In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 20.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 1.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [ ]:
import torch

BATCH_SIZE = 16

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

# Data Load

In [ ]:
def file_load(path):
    data_path = []
    f = open(path, 'r')
    while True:
        line = f.readline()
        if not line:
            break
        data_path.append(line[:-1])
    f.close()
    return data_path

In [ ]:
import torch.utils.data as data

class CustomDataset(data.Dataset):
    def __init__(self, opt):
        super(CustomDataset, self).__init__()
        dpath = 'drive/MyDrive/IIPL/p-transformer/europarl-v7.de-en'
        self.src = file_load('{0}.en_{1}.txt'.format(dpath, opt))
        self.trg = file_load('{0}.de_{1}.txt'.format(dpath, opt))

    def __getitem__(self, index):
        return self.src[index], self.trg[index]
    
    def __len__(self):
        return len(self.src)

In [ ]:
from collections import Counter
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data_iter, tokenizer):
    for text in data_iter:
        yield tokenizer(text)   # yield:제너레이터 반환

def build_vocab(data, tokenizer):
    vocab = build_vocab_from_iterator(yield_tokens(data, tokenizer), min_freq=10, specials=['<pad>','<unk>','<bos>','<eos>'])
    return vocab

In [ ]:
import torch
import torch.utils.data as data
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence


en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')

train_data = CustomDataset('train')
val_data = CustomDataset('val')
test_data = CustomDataset('test')

en_vocab = build_vocab(train_data.src, en_tokenizer)
de_vocab = build_vocab(train_data.trg, de_tokenizer)

In [ ]:
print(len(en_vocab), len(de_vocab))

27695 53888


In [ ]:
def _collate_fn(batch):
    src_list, trg_list = [], []

    for text in batch:
        processed_src = torch.tensor([en_vocab[token] if token in en_vocab else en_vocab['<unk>'] for token in en_tokenizer(text[0])])
        processed_trg = torch.tensor([de_vocab[token] if token in de_vocab else de_vocab['<unk>'] for token in de_tokenizer(text[1])])
        src_list.append(processed_src)
        trg_list.append(processed_trg)
    
    src_list = pad_sequence(src_list, batch_first=True, padding_value=0)
    trg_list = pad_sequence(trg_list, batch_first=True, padding_value=0)

    if src_list.size()[1] > 512:    # 토큰 길이 512로 자르기
        src_list = src_list.split(512, dim=1)[0]
    else:                           # 토큰 길이 512 되도록 패딩
        pad = torch.zeros(src_list.size()[0], 512 - src_list.size()[1], dtype=torch.int64)
        src_list = torch.cat([src_list, pad], dim=-1)
    
    if trg_list.size()[1] > 512:    # 토큰 길이 512로 자르기
        trg_list = trg_list.split(512, dim=1)[0]
    else:                           # 토큰 길이 512 되도록 패딩
        pad = torch.zeros(trg_list.size()[0], 512 - trg_list.size()[1], dtype=torch.int64)
        trg_list = torch.cat([trg_list, pad], dim=-1)

    return src_list, trg_list


class CustomDataLoader(data.DataLoader):
    def __init__(self, *args, **kwargs):
        super(CustomDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = _collate_fn

In [ ]:
train_iter = CustomDataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)
val_iter = CustomDataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_iter = CustomDataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
print(len(train_iter), len(val_iter), len(test_iter))

76809 19203 24003


# Utils

In [ ]:
import copy
import torch.nn as nn

def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# Model Architecture

## embedding

In [ ]:
import math
import torch.nn as nn

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

## generator

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=1)

## layer

In [ ]:
import torch
import torch.nn as nn

class MultiHeadedAttention(nn.Module):
    def __init__(self, n_heads, hid_dim, device, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert hid_dim % n_heads == 0
        self.head_dim = hid_dim // n_heads
        self.n_heads = n_heads
        self.hid_dim = hid_dim

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        self.dropout = nn.Dropout(p=dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask=None):

        batch_size = query.shape[0]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

        energy = torch.matmul(Q, K.permute(0,1,3,2)) / self.scale
        if mask is not None:
            energy = energy.masked_fill(mask==0, -1e10)
        attention = torch.softmax(energy, dim=-1)

        x = torch.matmul(self.dropout(attention),V)
        x = x.permute(0,2,1,3).contiguous()
        x = x.view(batch_size, -1, self.hid_dim)
        x = self.fc_o(x)

        return x, attention

In [ ]:
import torch.nn as nn

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [ ]:
import torch
import torch.nn as nn

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
import torch.nn as nn

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        drop = sublayer(self.norm(x))
        return x + self.dropout(drop[0])

## model

In [ ]:
class EncoderDecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(EncoderDecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.e_sublayer = clones(SublayerConnection(size, dropout), 2)
        self.d_sublayer = clones(SublayerConnection(size, dropout), 3)
 
    # encoder, decoder에 각각 input, output 있어야 하므로 input 2개, output 2개
    def forward(self, e_x, d_x, src_mask, tgt_mask):
        # EncoderLayer 파트
        e_x = self.e_sublayer[0](e_x, lambda e_x: self.self_attn(e_x, e_x, e_x, src_mask))
        e_x = self.e_sublayer[1](e_x, self.feed_forward)

        # DecoderLayer 파트
        d_x = self.d_sublayer[0](d_x, lambda d_x: self.self_attn(d_x, d_x, d_x, tgt_mask))
        d_x = self.d_sublayer[1](d_x, lambda d_x: self.src_attn(d_x, e_x, e_x, src_mask))     # 여기를 encoder layer의 output과 연결
        d_x = self.d_sublayer[2](d_x, self.feed_forward)

        return e_x, d_x

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, layer, N):
        super(EncoderDecoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, e_x, d_x, src_mask, tgt_mask):
        for layer in self.layers:
            e_x, d_x = layer(e_x, d_x, src_mask, tgt_mask)
        return self.norm(d_x)

In [ ]:
class pTransformer(nn.Module):
    def __init__(self, encoderdecoder, src_embed, tgt_embed, generator):
        super(pTransformer, self).__init__()
        self.encoderdecoder = encoderdecoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.generator(self.encoderdecoder(self.src_embed(src), self.tgt_embed(tgt), src_mask, tgt_mask))

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1, device='cpu'):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model, device)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = pTransformer(
        EncoderDecoder(EncoderDecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab)
    )

    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [ ]:
def subsequent_mask(size):
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


class Batch:
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(1).unsqueeze(2)
        if trg is not None:
            self.trg = trg
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        tgt_mask = (tgt != pad).unsqueeze(1).unsqueeze(2)
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [ ]:
import time

def run_epoch(data_iter, model, loss_compute, device):
    start = time.time()
    #total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, data in enumerate(data_iter):
        batch = Batch(data[0].to(device), data[1].to(device))
        out = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)       # out.size() : [batch_size, 512, vocab_size]
        out = torch.argmax(out, dim=2)      # vacab별 확률 중 가장 높은 것의 index 추출
        out = torch.where(batch.trg != 0, out, 0)

        out = [[de_vocab.get_itos()[token] for token in line] for line in out]
        trg = [[de_vocab.get_itos()[token] for token in line] for line in batch.trg]
        

        loss = loss_compute(out, [trg]+[[['No', 'Match']]]*(len(out)-1), 2, [0.5,0.5])
        total_loss += loss
        #total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 10 == 0:
            elapsed = time.time() - start
            print("Epoch Step: %d bleu_score: %f Tokens per Sec: %f" %
                    (i, loss, tokens / elapsed))
            start = time.time()
            tokens = 0
        
    return total_loss / len(data_iter)

In [ ]:
model = make_model(len(en_vocab), len(de_vocab), N=2, d_model=512, d_ff=1024, h=8, dropout=0.1, device=DEVICE)
model.to(DEVICE)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


pTransformer(
  (encoderdecoder): EncoderDecoder(
    (layers): ModuleList(
      (0): EncoderDecoderLayer(
        (self_attn): MultiHeadedAttention(
          (fc_q): Linear(in_features=512, out_features=512, bias=True)
          (fc_k): Linear(in_features=512, out_features=512, bias=True)
          (fc_v): Linear(in_features=512, out_features=512, bias=True)
          (fc_o): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (src_attn): MultiHeadedAttention(
          (fc_q): Linear(in_features=512, out_features=512, bias=True)
          (fc_k): Linear(in_features=512, out_features=512, bias=True)
          (fc_v): Linear(in_features=512, out_features=512, bias=True)
          (fc_o): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=1024, bias=Tru

In [ ]:
import numpy as np
from torchtext.data.metrics import bleu_score

EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()
    run_epoch(train_iter, model, bleu_score, DEVICE)

    model.eval()
    print('Epoch {0} val_bleu_score :'.format(epoch+1), run_epoch(val_iter, model, bleu_score, DEVICE))
    print()

Epoch Step: 0 bleu_score: 0.059445 Tokens per Sec: 5.101562
Epoch Step: 10 bleu_score: 0.056635 Tokens per Sec: 5.944981


KeyboardInterrupt: ignored